## Libraries

In [ ]:
from copy import deepcopy
from typing import Tuple, List

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from sys import maxsize as MAX_INT
import time

## Game State Representation

`Grid` class will represent the game state as a matrix

In [ ]:
class Grid:
    
    # initialization function
    def __init__(self, matrix):
        self.setMatrix(matrix)
    
    # function to check if two game state matrices are the same
    def __eq__(self, other) -> bool:
        for i in range(4):
            for j in range(4):
                if self.matrix[i][j] != other.matrix[i][j]:
                    return False
        return True
    
    # function to set matrix value
    def setMatrix(self, matrix):
        self.matrix = deepcopy(matrix)
    
    # function to get matrix value
    def getMatrix(self) -> List[List]:
        return deepcopy(self.matrix)
    
    # function to place tiles in the board matrix
    def placeTile(self, row: int, col: int, tile: int):
        self.matrix[row-1][col-1] = tile
    
    # function to get score of game state
    def utility(self) -> int:
        count = 0
        sum = 0
        for i in range(4):
            for j in range(4):
                sum += self.matrix[i][j]
                if self.matrix[i][j] != 0:
                    count += 1
        return int(sum/count)
    
    # function to check if algorithm can move 'up'
    def canMoveUp(self) -> bool:
        for j in range(4):
            k = -1
            for i in range(3, -1, -1):
                if self.matrix[i][j] > 0:
                    k = i
                    break
            if k > -1:
                for i in range(k, 0, -1):
                    if self.matrix[i-1][j] == 0 or self.matrix[i][j] == self.matrix[i-1][j]:
                        return True
        return False

    # function to check if algorithm can move 'down'
    def canMoveDown(self) -> bool:
        for j in range(4):
            k = -1
            for i in range(4):
                if self.matrix[i][j] > 0:
                    k = i
                    break
            if k > -1:
                for i in range(k, 3):
                    if self.matrix[i+1][j] == 0 or self.matrix[i][j] == self.matrix[i+1][j]:
                        return True
        return False

    # function to check if algorithm can move 'left'
    def canMoveLeft(self) -> bool:
        for i in range(4):
            k = -1
            for j in range(3, -1, -1):
                if self.matrix[i][j] > 0:
                    k = j
                    break
            if k > -1:
                for j in range(k, 0, -1):
                    if self.matrix[i][j-1] == 0 or self.matrix[i][j] == self.matrix[i][j-1]:
                        return True
        return False

    # function to check if algorithm can move 'right'
    def canMoveRight(self) -> bool:
        for i in range(4):
            k = -1
            for j in range(4):
                if self.matrix[i][j] > 0:
                    k = j
                    break
            if k > -1:
                for j in range(k, 3):
                    if self.matrix[i][j+1] == 0 or self.matrix[i][j] == self.matrix[i][j+1]:
                        return True
        return False
    
    # get possible move to make
    def getAvailableMovesForMax(self) -> List[int]:
        moves = []

        if self.canMoveUp():
            moves.append(0)
        if self.canMoveDown():
            moves.append(1)
        if self.canMoveLeft():
            moves.append(2)
        if self.canMoveRight():
            moves.append(3)
        
        return moves
    
    # get possible places where a new tile can be generated
    def getAvailableMovesForMin(self) -> List[Tuple[int]]:
        places = []
        for i in range(4):
            for j in range(4):
                if self.matrix[i][j] == 0:
                    places.append((i+1, j+1, 2))
                    places.append((i+1, j+1, 4))
        return places
    
    # get child state
    def getChildren(self, who: str) -> List:
        if who == "max":
            return self.getAvailableMovesForMax()
        elif who == "min":
            return self.getAvailableMovesForMin()
    
    # check if a game state is at the end
    def isTerminal(self, who: str) -> bool:
        if who == "max":
            if self.canMoveUp():
                return False
            if self.canMoveDown():
                return False
            if self.canMoveLeft():
                return False
            if self.canMoveRight():
                return False
            return True
        elif who == "min":
            for i in range(4):
                for j in range(4):
                    if self.matrix[i][j] == 0:
                        return False
            return True
    
    def isGameOver(self) -> bool:
        return self.isTerminal(who="max")
    
    # function to make the move 'up'
    def up(self):
        for j in range(4):
            w = 0
            k = 0
            for i in range(4):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[w][j] = 2*k
                    w += 1
                    k = 0
                else:
                    self.matrix[w][j] = k
                    w += 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[w][j] = k
                w += 1
            for i in range(w, 4):
                self.matrix[i][j] = 0
    
    # function to make the move 'down'
    def down(self):
        for j in range(4):
            w = 3
            k = 0
            for i in range(3, -1, -1):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[w][j] = 2*k
                    w -= 1
                    k = 0
                else:
                    self.matrix[w][j] = k
                    w -= 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[w][j] = k
                w -= 1
            for i in range(w+1):
                self.matrix[i][j] = 0
    
    # function to make the move 'left'
    def left(self):
        for i in range(4):
            w = 0
            k = 0
            for j in range(4):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[i][w] = 2*k
                    w += 1
                    k = 0
                else:
                    self.matrix[i][w] = k
                    w += 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[i][w] = k
                w += 1
            for j in range(w, 4):
                self.matrix[i][j] = 0
    
    # function to make the move 'right'
    def right(self):
        for i in range(4):
            w = 3
            k = 0
            for j in range(3, -1, -1):
                if self.matrix[i][j] == 0:
                    continue
                if k == 0:
                    k = self.matrix[i][j]
                elif k == self.matrix[i][j]:
                    self.matrix[i][w] = 2*k
                    w -= 1
                    k = 0
                else:
                    self.matrix[i][w] = k
                    w -= 1
                    k = self.matrix[i][j]
            if k != 0:
                self.matrix[i][w] = k
                w -= 1
            for j in range(w+1):
                self.matrix[i][j] = 0
    
    # wrapper function to make the move
    def move(self, mv: int) -> None:
        if mv == 0:
            self.up()
        elif mv == 1:
            self.down()
        elif mv == 2:
            self.left()
        else:
            self.right()
    
    # get the action to reach the child state
    def getMoveTo(self, child: 'Grid') -> int:
        if self.canMoveUp():
            g = Grid(matrix=self.getMatrix())
            g.up()
            if g == child:
                return 0
        if self.canMoveDown():
            g = Grid(matrix=self.getMatrix())
            g.down()
            if g == child:
                return 1
        if self.canMoveLeft():
            g = Grid(matrix=self.getMatrix())
            g.left()
            if g == child:
                return 2
        return 3

## Game Interaction

`GameDriver` class acts as a middleman between the game and the minimax algorithm, and thus responsible for interacting with the game in the browser

**Important!!** Ensure the path to the chrome webdriver is correct

In [ ]:
class GameDriver:  

    # initialization function  
    def __init__(self):
        self.url = 'https://play2048.co/'

        # the path to the chrome webdriver
        self.driver = webdriver.Chrome('chromedriver.exe')
        
        self.driver.get(self.url)
        self.body = self.driver.find_element_by_tag_name('body')
        self.moves = {
            0: Keys.ARROW_UP,
            1: Keys.ARROW_DOWN,
            2: Keys.ARROW_LEFT,
            3: Keys.ARROW_RIGHT
        }
    
    # get current game state from the browser, and return it as a `Grid` object
    def getGrid(self) -> Grid:
        matrix = [[0 for i in range(4)] for j in range(4)]
        tiles = self.driver.find_elements_by_class_name('tile')
        
        for tile in tiles:
            cls = tile.get_attribute('class')
            col, row = cls.split('tile-position-')[1].split(' ')[0].split('-')
            col, row = int(col)-1, int(row)-1
            num = int(cls.split('tile tile-')[1].split(' ')[0])
            if num > matrix[row][col]:
                matrix[row][col] = num
        
        return Grid(matrix)
    
    # emulate key press in the browser, based on the action passed as parameter
    def move(self, moveCode):
        self.body.send_keys(self.moves[moveCode])
        time.sleep(0.05)

## Min-Max Algorithm

with alpha-beta pruning

In [ ]:
# for "Max" player
def maximize(state: Grid, a: int, b: int, d: int) -> Tuple[Grid, int]:
    (maxChild, maxUtility) = (None, -1)

    if d == 0 or state.isTerminal(who="max"):
        return (None, state.utility())
    
    d -= 1
    
    for child in state.getChildren(who = "max"):
        grid = Grid(matrix=state.getMatrix())
        grid.move(child)
        (_, utility) = minimize(grid, a, b, d)
        if utility > maxUtility:
            (maxChild, maxUtility) = (grid, utility)
        if maxUtility >= b:
            break
        if maxUtility > a:
            a = maxUtility

    return (maxChild, maxUtility)

# for "Min" player
def minimize(state: Grid, a: int, b: int, d: int) -> Tuple[Grid, int]:
    (minChild, minUtility) = (None, MAX_INT)

    if d == 0 or state.isTerminal(who="min"):
        return (None, state.utility())

    d -= 1
    
    for child in state.getChildren(who = "min"):
        grid = Grid(matrix=state.getMatrix())
        grid.placeTile(child[0], child[1], child[2])
        (_, utility) = maximize(grid, a, b, d)
        if utility < minUtility:
            (minChild, minUtility) = (grid, utility)
        if minUtility <= a:
            break
        if minUtility < b:
            b = minUtility

    return (minChild, minUtility)

# get the move that maximizes the utility
def getBestMove(grid: Grid, depth: int = 5):
    (child, _) = maximize(Grid(matrix=grid.getMatrix()), -1, MAX_INT, depth)
    return grid.getMoveTo(child)

## Main

*   Get the game data
*   Use min-max to decide the best move
*   Perform the move
*   Repeat until end

In [ ]:
gameDriver = GameDriver()

moves_str = ['UP', 'DOWN', 'LEFT', 'RIGHT']
moves_count = 1

while True:
    grid = gameDriver.getGrid()

    if grid.isGameOver():
        print("Unfortunately, I lost the game.")
        break

    moveCode = getBestMove(grid, 5)

    print(f'Move #{moves_count}: {moves_str[moveCode]}')
    
    gameDriver.move(moveCode)
    moves_count += 1